# BTC Price Prediction with Linear Regression

This notebook trains a linear regression model to forecast Bitcoin closing price using the Fear & Greed Index and the previous closing price.

## Project Setup
Import required Python libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle
from datetime import datetime
import os
import sys

sys.path.append('./src')
from model_io import save_model

## Load and Explore Data
Load the processed dataset and perform basic checks.

In [ ]:
data_path = '../data/processed/fear_and_greed_history_5min.parquet'
df = pd.read_parquet(data_path)
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())

## Data Visualization
Visualize BTC price trends and its relationship with the Fear & Greed Index.

In [ ]:
plt.figure(figsize=(10,4))
sns.lineplot(data=df, x='interval_end_time', y='close_bitcoin_price_usd')
plt.title('BTC Close Price Over Time')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
sns.scatterplot(x='avg_actual_value', y='close_bitcoin_price_usd', data=df)
plt.title('Fear & Greed vs BTC Price')
plt.tight_layout()
plt.show()

plt.figure(figsize=(5,4))
sns.heatmap(df[['avg_actual_value','close_bitcoin_price_usd']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

## Preprocessing
Create features and split the data into training and testing sets.

In [ ]:
df['prev_close_price'] = df['close_bitcoin_price_usd'].shift(1)
df = df.dropna(subset=['prev_close_price'])

X = df[['prev_close_price', 'avg_actual_value']]
y = df['close_bitcoin_price_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

## Model Training
Train a linear regression model.

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

## Model Evaluation
Evaluate the model using Mean Squared Error and R² score.

In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse:.2f}')
print(f'R^2 Score: {r2:.4f}')

## Visualize Results
Compare actual vs predicted BTC prices.

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test, y=y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.title('Actual vs Predicted BTC Price')
plt.tight_layout()
plt.show()

## Model Versioning
Save the trained model with a timestamped filename.

In [ ]:
models_dir = '../models'
os.makedirs(models_dir, exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_path = os.path.join(models_dir, f'linear_model_{timestamp}.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

import polars as pl
weights_df = pl.DataFrame({
    'feature': ['intercept', 'prev_close_price', 'fear_greed_index'],
    'weight': [model.intercept_, *model.coef_]
})
msg = save_model(weights_df, mse, models_dir)
print(msg)
print(f'Model saved to {model_path}')